![image.png](plots/california_housing_intro.png)

El contenido de esta libreta está basado en el libro 

* [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/). Capítulo 2

Trabajaremos con un conjunto de datos que contiene información acerca de las casas en distintos distritos del estado de California. Entre otras variables, tendremos la población, los ingresos medianos del barrio, número medio de habitaciones por casa, etc. Con esta información, nuestro objetivo será intentar predecir el precio mediano de las casas de un barrio. Empecemos por cargar los paquetes básicos y acceder al conjunto de datos:

# Carga de datos
___

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(5) # Los resultados del notebook serán los mismos en cada ejecución

In [ ]:
housing = pd.read_csv('data/housing.csv')

housing.head()

In [ ]:
housing.info()

Tenemos un conjunto de datos con 11 variables. Una, `median_house_value` es la variable respuesta que queremos predecir, y de las otras 10, tenemos 1 categórca (`ocean_proximity`) y 9 numéricas. Ya que tenemos la `longitud` y `latitud` de los barrios del conjunto de datos, representemoslos gráficamente de forma que el color dependa de la variable respuesta.

# Visualización inicial
___

In [ ]:
p = housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)

Podemos ver claramente que el precio es mucho más alto en la costa, alrededor de Los Ángeles y San Francisco. Veamos los scatterplots entre la variable respuesta y las demás variables, para estudiar cuales pueden estar más relacionadas.

In [ ]:
var_names = [var for var in housing.columns if var != 'median_house_value']

plt.figure(figsize=(19,10))
for idx, var in enumerate(var_names):
    plt.subplot(3, 4, idx+1)
    plt.scatter(housing[var], housing['median_house_value'])
    plt.title(var)

Vemos que la variable más relacionada con la respuesta parece ser el `median_income`. Veamos también las correlaciones entre la variable respuesta y las demás variables:

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

Vemos que efectivamente, la variable más correlacionada es `median_income`, seguida (con una correlación negativa) por `bedrooms_per_household`. El que las correlaciones sean tan bajas nos hace prever que los resultados de nuestros modelos de predicción no serán muy buenos. A continuación veamos la relación entre la variable categórica `ocean_proximity` y la respuesta.

In [ ]:
housing['ocean_proximity'].value_counts().plot(kind='bar')

In [ ]:
from plotnine import *

(ggplot(data=housing, mapping=aes(x='ocean_proximity', y='median_house_value')) + geom_boxplot() + theme_light()) 

### <font color='D12828'> Ejercicio: </font>
1. Obtén un histograma de la variable respuesta `median_house_value`. Pista: usa la función `plt.hist`

In [ ]:
# Ejercicio


# Preprocesado
___

### <font color='D12828'> Ejercicio: </font>
1. Divide el dataframe `housing` en la parte de los predictores `X`, y la respuesta `y`. Pista: la variable respuesta es `median_house_value`
2. Divide `X` e `y` en un conjunto de train y otro de test de forma que el `train_size` sea de 5000 observaciones

In [ ]:
# Ejercicio


## Estandarización

Hemos visto que cuando tratamos con variables numéricas, normalmente es recomendable estandarizarlas para que la escala de las unidades no afecten a la capacidad de hacer predicciones de los algoritmos, pero solo tiene sentido estandarizar las variables que sean numéricas.

### <font color='D12828'> Ejercicio: </font>
1. Crea un dataframe llamado `X_num` solo con las variables numéricas de `X_train` (elimina `ocean_proximity`)
2. Estandariza el conjunto `X_num` usando la función `sklearn.preprocessing.StandardScaler`

In [ ]:
# Ejercicio


## Ona hot encoding

Por otro lado, tenemos la variable categórica `ocean_proximity`. La mayoría de algoritmos de sklearn no aceptan variables que no sean numéricas, por lo que para incorporar estas variables categóricas es necesario codificarlas. Para ello usaremos la conocida como __one-hot encoding__, utilizada en prácticamente todos los modelos de ML, directa o indirectamente, para incorporar variables categóricas:

![image.png](plots/one_hot_encoding.png)

La idea es sencilla: dada una variable categórica con k clases, se construyen k variables numéricas que codifican con 1 o 0 cada clase. Esta codificación ya está programada en sklearn, veamos como usarla:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

X_cat = X_train[["ocean_proximity"]]

cat_encoder = OneHotEncoder()
X_cat_onehot = cat_encoder.fit_transform(X_cat)

print(X_cat_onehot.toarray()[0:5, 0:5], '\n')

print(cat_encoder.categories_)

## Preprocesado con pipelines

Ahora estamos haciendo sobre una parte de las variables (las numéricas) un tipo de preprocesado, y sobre otra parte de las variables (la categórica) otro preprocesado. Veamos cómo podemos unificar todo este proceso usando un tipo especial de pipeline llamado `ColumnTransformer`, que nos permite aplicar funciones diferentes a columnas diferentes de los datos:

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = [var for var in X_train if var != 'ocean_proximity']
cat_attribs = ['ocean_proximity']

print(num_attribs, '\n')
print(cat_attribs)

In [ ]:

full_pipeline = ColumnTransformer([
    # Preprocesado sobre variables numéricas
    ('num', StandardScaler(), num_attribs),
    
    # Preprocesado sobre variables categóricas
    ('cat', OneHotEncoder(), cat_attribs),
])

X_train_preproces = full_pipeline.fit_transform(X_train)

La ventaja principal de construir un pipeline de este estilo viene sobre todo cuando necesitamos llevar a cabo el mismo preprocesado más de una vez, por ejemplo, sobre el conjunto de datos de test, o al recibir nuevas observaciones sobre las que queramos hacer predicciones.

### <font color='D12828'> Ejercicio: </font>
1. Lleva a cabo el preprocesado del conjunto de datos `X_test` y almacena el resultado en `X_test_preproces`. Recuerda que queremos escalar los datos del test en base a la información del train.

In [ ]:
# Ejercicio


# Construcción del modelo
___

Ahora que ya tenemos el conjunto de datos preparado, es hora de construir el modelo. Dado que la variable respuesta es numérica, se trata de un problema de regresión, por lo que construiremos una SVM de regresión.

In [ ]:
from sklearn.svm import SVR

# Construcción del modelo
svr_lineal = SVR(kernel='linear')
svr_lineal.fit(X_train_preproces, y_train)

# Predicción 
prediccion = svr_lineal.predict(X_test_preproces)

print(f'Predicción {np.round(prediccion[11:14], 0)}')
print(f'Valor      {y_test[11:14].values}')


Veamos una métrica del error de este modelo. Para ello usaremos el `mean_squared_error`, o error cuadrático medio. Este error calcula la diferencia entre las predicciones y el verdadero valor de las observaciones, y después las eleva al cuadrado. Al elevarlas al cuadrado se consigue que todos los errores sean números positivos, y por último hace la media de los errores. Es muy habitual tomar después la raiz de este error,   para devolver el error a las unidades originales (es el llamado root mean squared error), ya que es más sencillo de interpretar:

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, prediccion)
rmse = np.sqrt(mse)

print(f'RMSE: {rmse}')

### <font color='D12828'> Ejercicio: </font>
1. Intentemos mejorar este modelo. Utiliza la función de `sklearn.model_selection.GridSearchCV` para optimizar el tipo de red. En el mallado, incluye una red lineal y optimiza `C`, y una red rbf y optimiza `C` y `gamma`. Utiliza como scoring `neg_mean_squared_error`
2. Obtén el RMSE de las predicciones en la parte de test.

In [ ]:
# Ejercicio


# Final
___
Con esto termina nuestro análisis del conjunto de datos de MNIST.

![image.png](plots/fin_svm_2.png)